In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

batch_size = 128
num_classes = 10
epochs = 50
num_channels = 1

# input image dimensions
img_rows, img_cols = 28, 28


def convert_images(raw):
    global img_rows
    global img_cols
    global num_channels
    images = raw.reshape([-1, num_channels, img_rows, img_cols])
    # Reorder the indices of the array.
    images = images.transpose([0, 2, 3, 1])
    return images


train = pd.read_csv('train.csv')
# the first character in the train set is the label, break it off
labels = train.iloc[:, 0]
features = train.iloc[:, 1:785]

test = pd.read_csv('test.csv')
test /= 255
kaggleTest = test.values.reshape(test.shape[0], 28, 28, 1)

x_train, x_test, y_train, y_test = train_test_split(
    features.values, labels.values, test_size=0.2, random_state=1212)

x_train = convert_images(x_train)
x_test = convert_images(x_test)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(img_rows, img_cols, num_channels)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


kagglePred = model.predict(kaggleTest)
results = np.argmax(kagglePred, axis=1)
results = pd.Series(results, name="Label")

submission = pd.concat([pd.Series(range(1, 28001), name="ImageId"), results], axis=1)

submission.to_csv("submission.csv", index=False)


x_train shape: (33600, 28, 28, 1)
33600 train samples
8400 test samples
Train on 33600 samples, validate on 8400 samples
Epoch 1/50
33600/33600 [==============================] - 3s 104us/sample - loss: 0.3442 - acc: 0.8913 - val_loss: 0.0880 - val_acc: 0.9731
Epoch 2/50
33600/33600 [==============================] - 3s 95us/sample - loss: 0.1117 - acc: 0.9670 - val_loss: 0.0602 - val_acc: 0.9806
Epoch 3/50
33600/33600 [==============================] - 3s 94us/sample - loss: 0.0808 - acc: 0.9752 - val_loss: 0.0524 - val_acc: 0.9835
Epoch 4/50
33600/33600 [==============================] - 3s 95us/sample - loss: 0.0659 - acc: 0.9801 - val_loss: 0.0482 - val_acc: 0.9855
Epoch 5/50
33600/33600 [==============================] - 3s 95us/sample - loss: 0.0542 - acc: 0.9836 - val_loss: 0.0413 - val_acc: 0.9879
Epoch 6/50
33600/33600 [==============================] - 3s 95us/sample - loss: 0.0485 - acc: 0.9853 - val_loss: 0.0423 - val_acc: 0.9880
Epoch 7/50
33600/33600 [====================